In [2]:
# 그룹별 집계
# 판다스의 또 다른 중요한 기능 중 하나는 그룹별 집계(groupby)
    # 데이터로부터 동일한 객체를 가진 데이터만 따로 뽑아 기술통계 데이터를 추출하는 기능

# groupby 명령어는 '분할 -> 적용 -> 결합'과정을 거침
    # 분할(split) : 같은 종류의 데이터끼리 나누는 기능
        # 데이터를 나누는 기준은 키 값이 되며, 이 키 값은 다양한 종류의 열이 될 수 있음
    # 적용(apply) : 데이터 분할로 나눠진 블록마다 다양한 연산 적용
        # 판다스에서는 연산 적용을 위한 함수를 분석가 개인이 직접 만들어 사용 가능
    # 결합(combine) : 나눠진 블록마다 연산 적용으로 인해 함수가 적용된 각 블록들을 합치는 단계
        # 최종적으로 어떤 키 값을 기준으로 통합된 통계 데이터를 추출 가능

In [3]:
# 그룹별 집계의 기본형

import pandas as pd
import numpy as np

ipl_data = {'Team' : ['Riders', 'Riders', 'Devils', 'Devils', 'Kings', 'Kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
            'Rank' : [1, 2, 2, 3, 3, 4, 1, 1, 2, 4, 1, 2],
            'Year' : [2014, 2015, 2014, 2015, 2014, 2015, 2016, 2017, 2016, 2014, 2015, 2017],
            'Points' : [876, 789, 863, 673, 741, 812, 756, 788, 694, 701, 804, 690]}

df = pd.DataFrame(ipl_data)
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [4]:
# 간단하게 그룹별 집계 사용 가능

df.groupby('Team')['Points'].sum()
# Team이라는 키 값으로 데이터를 묶음(split)
# Points 열을 추출한 다음 sum 함수를 적용(apply)
# 그 결과를 데이터 분석가에게 결합(combine)하여 나타냄

Team
Devils    1536
Kings     3097
Riders    3049
Royals    1505
Name: Points, dtype: int64

In [5]:
# 멀티 인덱스 그룹별 집계
# 실제로는 한 개 이상의 열을 기준으로 그룹별 집계를 실행하는 경우가 많음
# 한 개 이상의 열을 넣어야 할 때에는 리스트를 사용
    # 이 경우 한 개 이상의 열이 키 값으로 표현되는 계층적 인덱스(hierarchical index) 형태로 결과 출력

multi_groupby = df.groupby(['Team', 'Year'])['Points'].sum()
multi_groupby

# 이렇게 출력된 결과물 역시 시리즈(Series) 객체
    # 이를 다시 데이터프레임 형태로 변형하기 위해서는 reset_index 함수 적용

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2015    812
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
Name: Points, dtype: int64

In [6]:
# 멀티 인덱스
    # 그룹별 집계를 하면 한 개 이상의 인덱스를 갖는 시리즈 객체가 출력
        # 한 개 이상의 열을 사용해 그룹별 집계를 수행하면 해당 열이 모두 인덱스로 변환 => 멀티 인덱스
# 멀티 인덱스는 다음 코드와 같이 인덱스 요소(index property)를 이용해서 확인 가능

multi_groupby = df.groupby(['Team', 'Year'])['Points'].sum()
multi_groupby.index

MultiIndex([('Devils', 2014),
            ('Devils', 2015),
            ( 'Kings', 2014),
            ( 'Kings', 2015),
            ( 'Kings', 2016),
            ( 'Kings', 2017),
            ('Riders', 2014),
            ('Riders', 2015),
            ('Riders', 2016),
            ('Riders', 2017),
            ('Royals', 2014),
            ('Royals', 2015)],
           names=['Team', 'Year'])

In [7]:
# 멀티 인덱스 기능에서 인덱스의 추출을 유용하게 사용 가능

multi_groupby['Devils' : 'Kings']
# :(콜론)은 슬라이싱, 인덱싱에서 쓰이는 것과 같은 의미

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2015    812
        2016    756
        2017    788
Name: Points, dtype: int64

In [8]:
# unstack 함수 사용 => 기존 인덱스를 기준으로 묶인 데이터에서 두 번째 인덱스를 열로 변화
    # 엑셀의 피봇테이블과 유사한 형태로 데이터를 보여줌

multi_groupby.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,NaN,NaN
Kings,741.0,812.0,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,NaN,NaN


In [9]:
# 필요에 따라 swaplevel 함수를 사용하면 인덱스 간 레벨 변경 가능
# sort_index 함수를 사용하면 변경된 인덱스 중 첫 번째 인덱스를 기준으로 데이터 재정렬

multi_groupby.swaplevel().sort_index()

Year  Team  
2014  Devils    863
      Kings     741
      Riders    876
      Royals    701
2015  Devils    673
      Kings     812
      Riders    789
      Royals    804
2016  Kings     756
      Riders    694
2017  Kings     788
      Riders    690
Name: Points, dtype: int64

In [10]:
# 각 레벨 별로 별도의 연산 함수 적용 가능

teamlevel = multi_groupby.groupby(level= 0)
yearlevel = multi_groupby.groupby(level= 1)
teamlevel.sum(), yearlevel.sum()

(Team
 Devils    1536
 Kings     3097
 Riders    3049
 Royals    1505
 Name: Points, dtype: int64,
 Year
 2014    3181
 2015    3078
 2016    1450
 2017    1478
 Name: Points, dtype: int64)

In [11]:
multi_groupby.groupby(level=0).sum(), multi_groupby.groupby(level=1).sum()

(Team
 Devils    1536
 Kings     3097
 Riders    3049
 Royals    1505
 Name: Points, dtype: int64,
 Year
 2014    3181
 2015    3078
 2016    1450
 2017    1478
 Name: Points, dtype: int64)

In [12]:
# 그룹화된 상태
# 분할 -> 적용 -> 결합으로 이어지는 groupby 함수사용에서 분할만 적용한 상태로 데이터를 다룰수 있음
    # 그룹화된(grouped) 상태

# 그룹화된 상태에서 get_group 함수를 사용하면 해당 키값을 기준으로 분할된 데이터프레임 객체를 확인 가능

grouped = df.groupby("Team")
grouped.get_group("Riders")

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
8,Riders,2,2016,694
11,Riders,2,2017,690


In [14]:
# 그룹화된 상태의데이터프레임에 다음과 같은 기능 적용 가능
    # 집계(aggregation): 요약된 통계 정보를 추출
    # 변환(transformation) : 해당 정보를 변환
    # 필터(filter) : 특정 정보를 제거하여 보여주는 필터링

In [16]:
# 집계
# agg 함수 사용 : 다양한 함수를 그대로 적용 => 단순 groupby를 사용할 때의 결과와 크게 다르지 않음

grouped.agg(min)

,Rank,Year,Points
Team,,,
Devils,2,2014,673
Kings,1,2014,741
Riders,1,2014,690
Royals,1,2014,701


In [17]:
grouped.agg(np.mean)

,Rank,Year,Points
Team,,,
Devils,2.50,2014.5,768.00
Kings,2.25,2015.5,774.25
Riders,1.75,2015.5,762.25
Royals,2.50,2014.5,752.50


In [18]:
# 변환
# 집계와 달리 키 값별로 요약된 정보가 아닌 개별 데이터의 변환 지원
    # 적용하는 시범에서는 그룹화된 상태의 값으로 적용
        # max 함수를 사용하면 해당 그룹화된 상태에서 max 값들을 추출해 넣은 후 결합(combine)이 일어남
# 그룹화된 상태에서 통계 정보가 필요할 때 transform 함수 사용하기
    # 이 경우 그룹화된 상태의 평균값과 표준편차를 추출하여 각각 분할(split)된 상태의 값에 적용시킴

grouped.transform(max)

,Rank,Year,Points
0,2,2017,876
1,2,2017,876
2,3,2015,863
3,3,2015,863
4,4,2017,812
5,4,2017,812
6,4,2017,812
7,4,2017,812
8,2,2017,876
9,4,2015,804


In [19]:
score = lambda x : (x - x.mean()) / x.std()
grouped.transform(score)

,Rank,Year,Points
0,-1.500000,-1.161895,1.284327
1,0.500000,-0.387298,0.302029
2,-0.707107,-0.707107,0.707107
3,0.707107,0.707107,-0.707107
4,0.500000,-1.161895,-1.042333
5,1.166667,-0.387298,1.183401
6,-0.833333,0.387298,-0.572108
7,-0.833333,1.161895,0.431040
8,0.500000,0.387298,-0.770596
9,0.707107,-0.707107,-0.707107


In [21]:
# 필터
# 필터는 특정 조건으로 데이터를 검색할 때 사용하는 함수
# filter 함수 사용... x가 의미하는 건 분할된 상태에서 각각의 그룹화된 데이터프레임을 의미

df.groupby('Team').filter(
    lambda x : len(x) >= 3)

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
11,Riders,2,2017,690


In [22]:
# Points 열 추출
    # filter 함수 안에 있는 lambda 함수가 분할된 데이터프레임 전체를 입력 매개변수로 받기 때문

df.groupby('Team').filter(
    lambda x :
    x['Points'].max() > 800)

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701
